#Salvando uma rede neural

In [1]:
import pandas as pd
from google.colab import files

uploaded =files.upload()

Saving entradas_breast.csv to entradas_breast.csv


In [2]:
import io
  
previsores = pd.read_csv(io.BytesIO(uploaded['entradas_breast.csv']))

In [3]:
uploaded =files.upload()

Saving saidas_breast.csv to saidas_breast.csv


In [4]:
classe = pd.read_csv(io.BytesIO(uploaded['saidas_breast.csv']))

As validações utilizando as bases de teste foram feitas nos projetos anteriores portanto agora os melhores valores são colocados nos respectivos valores

In [5]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

classificador=Sequential()

#Primeira camada oculta
classificador.add(Dense(units =8, activation= 'relu',
                        kernel_initializer='normal', input_dim=30)) 

#camada de dropout, passando o parametro da porcentagem que eu quero zerar
classificador.add(Dropout(0.2))

#Segunda camada oculta  
classificador.add(Dense(units =8, activation= 'relu', 
                        kernel_initializer='normal')) 
classificador.add(Dropout(0.2))

#camada de saida
classificador.add(Dense(units =1, activation= 'sigmoid')) 

classificador.compile(optimizer='adam', loss='binary_crossentropy', 
                      metrics=['binary_accuracy'])

classificador.fit(previsores,classe,batch_size=10, epochs=100)

Epoch 1/100
57/57 [==============================] - 1s 2ms/step - loss: 2.7722 - binary_accuracy: 0.5202
Epoch 2/100
57/57 [==============================] - 0s 2ms/step - loss: 0.7151 - binary_accuracy: 0.6011
Epoch 3/100
57/57 [==============================] - 0s 2ms/step - loss: 0.9112 - binary_accuracy: 0.6292
Epoch 4/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6275 - binary_accuracy: 0.6204
Epoch 5/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6148 - binary_accuracy: 0.6098
Epoch 6/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6750 - binary_accuracy: 0.6151
Epoch 7/100
57/57 [==============================] - 0s 2ms/step - loss: 0.5870 - binary_accuracy: 0.6380
Epoch 8/100
57/57 [==============================] - 0s 2ms/step - loss: 0.5815 - binary_accuracy: 0.6485
Epoch 9/100
57/57 [==============================] - 0s 2ms/step - loss: 0.5337 - binary_accuracy: 0.6731
Epoch 10/100
57/57 [==========================

Salvando o classificador em disco


In [6]:
classificador_json=classificador.to_json()

In [7]:
classificador_json

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 30], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 30], "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomNormal", "config": {"mean": 0.0, "stddev": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout", "trainable": true, "dtype": "float32", "rate": 0.2, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "k

In [11]:
with open('classificador_breast.json','w') as json_file:
  json_file.write(classificador_json)

Salvar os pesos

In [9]:
classificador.save_weights('classificador_breast.h5')

#Carregando uma rede neural

In [23]:
from keras.models import model_from_json
arquivo=open('classificador_breast.json','r')
estrutura_rede=arquivo.read()
arquivo.close()

In [24]:
classificador=model_from_json(estrutura_rede)
classificador.load_weights('classificador_breast.h5')

Testando em um registro

In [25]:
import numpy as np
novo = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])

previsao = classificador.predict(novo)
previsao

array([[0.807923]], dtype=float32)

In [26]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

In [27]:
previsores=pd.read_csv('entradas_breast.csv')
classe=pd.read_csv('saidas_breast.csv')
classificador.compile(loss='binary_crossentropy', optimizer='adam',metrics=['binary_accuracy'])

In [28]:
resultado=classificador.evaluate(previsores,classe)
resultado

18/18 [==============================] - 0s 1ms/step - loss: 0.3357 - binary_accuracy: 0.8576


[0.3356555998325348, 0.8576449751853943]